In [1]:
import os
import pandas as pd
import shutil
from pathlib import Path

# ============================================
# CONFIGURATION
# ============================================

# Chemin vers vos dossiers d'utilisateurs (001 à 025)
SOURCE_DIR = r"C:\Users\sersi\Desktop\projet_SE_et_IOT\ASL-Sensor-Dataglove-Dataset"

# Dossier de sortie pour l'organisation par lettre/mot
OUTPUT_DIR = r"C:\Users\sersi\Desktop\projet_SE_et_IOT\ASL-Sensor-Dataglove-Dataset\dataset_par_lettre"

# Colonnes à garder pour le modèle ML
COLUMNS_TO_KEEP = [
    'flex_1', 'flex_2', 'flex_3', 'flex_4', 'flex_5',  # Flex sensors
    'GYRx', 'GYRy', 'GYRz',                            # Gyroscope
    'ACCx', 'ACCy', 'ACCz'                             # Accelerometer
]

# ============================================
# FONCTION 1: RÉORGANISER PAR LETTRE/MOT
# ============================================

def reorganize_by_letter_or_word():
    """
    Réorganise les fichiers CSV par lettre/mot
    Structure finale: dataset_par_lettre/A/a_001.csv, a_002.csv, etc.
    """
    
    # Créer le dossier de sortie
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    # Liste pour suivre les statistiques
    stats = {}
    
    # Parcourir tous les dossiers d'utilisateurs (001 à 025)
    for user_folder in sorted(os.listdir(SOURCE_DIR)):
        user_path = os.path.join(SOURCE_DIR, user_folder)
        
        # Vérifier que c'est un dossier et qu'il correspond au pattern (001-025)
        if not os.path.isdir(user_path) or not user_folder.isdigit():
            continue
            
        print(f"📂 Traitement du dossier: {user_folder}")
        
        # Parcourir tous les fichiers CSV dans le dossier utilisateur
        for csv_file in os.listdir(user_path):
            if not csv_file.endswith('.csv'):
                continue
                
            # Extraire le nom de la lettre/mot (ex: "a" de "a.csv")
            letter_or_word = csv_file.replace('.csv', '')
            
            # Créer un dossier pour cette lettre/mot (en majuscule)
            letter_folder = os.path.join(OUTPUT_DIR, letter_or_word.upper())
            os.makedirs(letter_folder, exist_ok=True)
            
            # Chemin source et destination
            source_file = os.path.join(user_path, csv_file)
            dest_file = os.path.join(letter_folder, f"{letter_or_word}_{user_folder}.csv")
            
            # Copier le fichier
            shutil.copy2(source_file, dest_file)
            
            # Mettre à jour les statistiques
            if letter_or_word not in stats:
                stats[letter_or_word] = 0
            stats[letter_or_word] += 1
            
            print(f"  ✅ Copié: {csv_file} → {letter_or_word.upper()}/{letter_or_word}_{user_folder}.csv")
    
    # Afficher les statistiques
    print("\n" + "="*50)
    print("📊 STATISTIQUES DE RÉORGANISATION")
    print("="*50)
    for letter, count in sorted(stats.items()):
        print(f"{letter:15s}: {count:3d} fichiers")
    print(f"\n✨ Total: {sum(stats.values())} fichiers copiés")


# ============================================
# FONCTION 2: NETTOYER ET PRÉPARER POUR ML
# ============================================

def prepare_for_ml(combine_all=False):
    """
    Nettoie les données et garde uniquement les colonnes nécessaires
    
    Args:
        combine_all: Si True, combine tous les utilisateurs en un seul fichier par lettre/mot
                     Si False, garde les fichiers séparés mais les nettoie
    """
    
    output_ml_dir = os.path.join(SOURCE_DIR, "dataset_ml_ready")
    os.makedirs(output_ml_dir, exist_ok=True)
    
    # Parcourir tous les dossiers de lettres/mots
    for letter_folder in os.listdir(OUTPUT_DIR):
        letter_path = os.path.join(OUTPUT_DIR, letter_folder)
        
        if not os.path.isdir(letter_path):
            continue
            
        print(f"\n🔧 Traitement de: {letter_folder}")
        
        if combine_all:
            # Combiner tous les fichiers en un seul
            all_data = []
            
            for csv_file in os.listdir(letter_path):
                if not csv_file.endswith('.csv'):
                    continue
                    
                file_path = os.path.join(letter_path, csv_file)
                
                try:
                    # Lire le CSV
                    df = pd.read_csv(file_path)
                    
                    # Garder uniquement les colonnes nécessaires
                    df_clean = df[COLUMNS_TO_KEEP].copy()
                    
                    # Ajouter la colonne label
                    df_clean['label'] = letter_folder.lower()
                    
                    # Ajouter à la liste
                    all_data.append(df_clean)
                    
                except Exception as e:
                    print(f"  ⚠️  Erreur avec {csv_file}: {e}")
                    continue
            
            # Combiner tous les DataFrames
            if all_data:
                combined_df = pd.concat(all_data, ignore_index=True)
                
                # Sauvegarder
                output_file = os.path.join(output_ml_dir, f"{letter_folder.lower()}_combined.csv")
                combined_df.to_csv(output_file, index=False)
                
                print(f"  ✅ Fichier combiné créé: {output_file}")
                print(f"     Lignes: {len(combined_df)}, Colonnes: {len(combined_df.columns)}")
        
        else:
            # Garder les fichiers séparés mais les nettoyer
            letter_ml_folder = os.path.join(output_ml_dir, letter_folder)
            os.makedirs(letter_ml_folder, exist_ok=True)
            
            for csv_file in os.listdir(letter_path):
                if not csv_file.endswith('.csv'):
                    continue
                    
                file_path = os.path.join(letter_path, csv_file)
                
                try:
                    # Lire le CSV
                    df = pd.read_csv(file_path)
                    
                    # Garder uniquement les colonnes nécessaires
                    df_clean = df[COLUMNS_TO_KEEP].copy()
                    
                    # Ajouter la colonne label
                    df_clean['label'] = letter_folder.lower()
                    
                    # Sauvegarder
                    output_file = os.path.join(letter_ml_folder, csv_file)
                    df_clean.to_csv(output_file, index=False)
                    
                    print(f"  ✅ Nettoyé: {csv_file}")
                    
                except Exception as e:
                    print(f"  ⚠️  Erreur avec {csv_file}: {e}")
                    continue


# ============================================
# FONCTION 3: CRÉER UN DATASET UNIQUE
# ============================================

def create_single_dataset():
    """
    Crée un seul fichier CSV contenant TOUTES les données
    Format: [flex_1...flex_5, GYRx...GYRz, ACCx...ACCz, label]
    """
    
    print("\n🔨 Création du dataset unique...")
    
    all_data = []
    
    # Parcourir tous les dossiers d'utilisateurs
    for user_folder in sorted(os.listdir(SOURCE_DIR)):
        user_path = os.path.join(SOURCE_DIR, user_folder)
        
        if not os.path.isdir(user_path) or not user_folder.isdigit():
            continue
        
        # Parcourir tous les fichiers CSV
        for csv_file in os.listdir(user_path):
            if not csv_file.endswith('.csv'):
                continue
                
            letter_or_word = csv_file.replace('.csv', '')
            file_path = os.path.join(user_path, csv_file)
            
            try:
                # Lire le CSV
                df = pd.read_csv(file_path)
                
                # Garder uniquement les colonnes nécessaires
                df_clean = df[COLUMNS_TO_KEEP].copy()
                
                # Ajouter les colonnes metadata
                df_clean['label'] = letter_or_word
                df_clean['user_id'] = user_folder
                
                # Ajouter à la liste
                all_data.append(df_clean)
                
            except Exception as e:
                print(f"  ⚠️  Erreur avec {user_folder}/{csv_file}: {e}")
                continue
    
    # Combiner tous les DataFrames
    if all_data:
        final_dataset = pd.concat(all_data, ignore_index=True)
        
        # Sauvegarder
        output_file = os.path.join(SOURCE_DIR, "asl_dataset_complete.csv")
        final_dataset.to_csv(output_file, index=False)
        
        print(f"\n✅ Dataset unique créé: {output_file}")
        print(f"   📊 Total de lignes: {len(final_dataset):,}")
        print(f"   📊 Colonnes: {list(final_dataset.columns)}")
        print(f"   📊 Labels uniques: {final_dataset['label'].nunique()}")
        print(f"   📊 Utilisateurs: {final_dataset['user_id'].nunique()}")
        
        # Afficher la répartition par label
        print("\n📈 Répartition par label:")
        print(final_dataset['label'].value_counts().sort_index())
        
        return final_dataset
    
    return None


# ============================================
# FONCTION PRINCIPALE
# ============================================

def main():
    """
    Fonction principale - exécute toutes les étapes
    """
    
    print("="*60)
    print("🚀 RÉORGANISATION DU DATASET ASL")
    print("="*60)
    
    # Étape 1: Réorganiser par lettre/mot
    print("\n📁 ÉTAPE 1: Réorganisation par lettre/mot")
    print("-"*60)
    reorganize_by_letter_or_word()
    
    # Étape 2: Préparer pour ML (fichiers séparés)
    print("\n🧹 ÉTAPE 2: Nettoyage pour ML (fichiers séparés)")
    print("-"*60)
    prepare_for_ml(combine_all=False)
    
    # Étape 3: Créer dataset combiné par lettre
    print("\n📦 ÉTAPE 3: Création de datasets combinés par lettre")
    print("-"*60)
    prepare_for_ml(combine_all=True)
    
    # Étape 4: Créer un dataset unique
    print("\n🎯 ÉTAPE 4: Création du dataset unique complet")
    print("-"*60)
    create_single_dataset()
    
    print("\n" + "="*60)
    print("✨ TRAITEMENT TERMINÉ !")
    print("="*60)
    print("\n📂 Dossiers créés:")
    print("   1. dataset_par_lettre/     - Organisé par lettre/mot")
    print("   2. dataset_ml_ready/       - Nettoyé pour ML")
    print("   3. asl_dataset_complete.csv - Dataset unique complet")


# ============================================
# EXÉCUTION
# ============================================

if __name__ == "__main__":
    main()

🚀 RÉORGANISATION DU DATASET ASL

📁 ÉTAPE 1: Réorganisation par lettre/mot
------------------------------------------------------------
📂 Traitement du dossier: 001
  ✅ Copié: a.csv → A/a_001.csv
  ✅ Copié: b.csv → B/b_001.csv
  ✅ Copié: bad.csv → BAD/bad_001.csv
  ✅ Copié: c.csv → C/c_001.csv
  ✅ Copié: d.csv → D/d_001.csv
  ✅ Copié: deaf.csv → DEAF/deaf_001.csv
  ✅ Copié: e.csv → E/e_001.csv
  ✅ Copié: f.csv → F/f_001.csv
  ✅ Copié: fine.csv → FINE/fine_001.csv
  ✅ Copié: g.csv → G/g_001.csv
  ✅ Copié: good.csv → GOOD/good_001.csv
  ✅ Copié: goodbye.csv → GOODBYE/goodbye_001.csv
  ✅ Copié: h.csv → H/h_001.csv
  ✅ Copié: hello.csv → HELLO/hello_001.csv
  ✅ Copié: hungry.csv → HUNGRY/hungry_001.csv
  ✅ Copié: i.csv → I/i_001.csv
  ✅ Copié: j.csv → J/j_001.csv
  ✅ Copié: k.csv → K/k_001.csv
  ✅ Copié: l.csv → L/l_001.csv
  ✅ Copié: m.csv → M/m_001.csv
  ✅ Copié: me.csv → ME/me_001.csv
  ✅ Copié: n.csv → N/n_001.csv
  ✅ Copié: no.csv → NO/no_001.csv
  ✅ Copié: o.csv → O/o_001.csv
  ✅ Copi